In [1]:
#necessary libraries
import re
import chardet 
import operator
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud

from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import Dropout, SpatialDropout1D

from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv('/content/Ben_fulltrain.csv')

In [3]:
data_test = pd.read_csv('/content/trac2_iben_dev.csv')

In [4]:
data_test.head()

ID  ... Sub-task B
0  C59.2078  ...       NGEN
1   C68.537  ...       NGEN
2  C59.1344  ...       NGEN
3   C59.706  ...        GEN
4   C68.663  ...       NGEN

[5 rows x 4 columns]

In [6]:
data 

Unnamed: 0                                               Text  label
0              0  কবির সিংহ একেবারে জ্ঞানহীন সিনেমা .... এখানকার...      2
1              1  @ তরুন ভার্মা আপনি কোন মেয়েকে দেখেছেন যে আমি ...      2
2              2  কিছু লোক ভাল জিনিস পছন্দ করেন না তাদের সম্পর্ক...      2
3              3              ভাই আপনি এই প্রজন্মের একজন ত্রাণকর্তা      2
4              4  আমি নারীবাদী নই ... L n আমি লাস্ট স্টোরিগুলি ঘ...      2
...          ...                                                ...    ...
8076        7499                                 rani mondal ekta ։      2
8077        7500  যেইসব শযোর ঈদ এর দিন রেপ তাদের আবার আলাহ क सथ...      2
8078        7501             কততার মতো to খানকির মেযেকে বেযাদব and      2
8079        7502                                        baja maya ।      2
8080        7503                   ore magi tor a vikkha korte hobe      2

[8081 rows x 3 columns]

In [7]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [8]:
data

Text  label
0     কবির সিংহ একেবারে জ্ঞানহীন সিনেমা .... এখানকার...      2
1     @ তরুন ভার্মা আপনি কোন মেয়েকে দেখেছেন যে আমি ...      2
2     কিছু লোক ভাল জিনিস পছন্দ করেন না তাদের সম্পর্ক...      2
3                 ভাই আপনি এই প্রজন্মের একজন ত্রাণকর্তা      2
4     আমি নারীবাদী নই ... L n আমি লাস্ট স্টোরিগুলি ঘ...      2
...                                                 ...    ...
8076                                 rani mondal ekta ։      2
8077  যেইসব শযোর ঈদ এর দিন রেপ তাদের আবার আলাহ क सथ...      2
8078             কততার মতো to খানকির মেযেকে বেযাদব and      2
8079                                        baja maya ।      2
8080                   ore magi tor a vikkha korte hobe      2

[8081 rows x 2 columns]

In [5]:
data_test.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [9]:
data.head()

Text  label
0  কবির সিংহ একেবারে জ্ঞানহীন সিনেমা .... এখানকার...      2
1  @ তরুন ভার্মা আপনি কোন মেয়েকে দেখেছেন যে আমি ...      2
2  কিছু লোক ভাল জিনিস পছন্দ করেন না তাদের সম্পর্ক...      2
3              ভাই আপনি এই প্রজন্মের একজন ত্রাণকর্তা      2
4  আমি নারীবাদী নই ... L n আমি লাস্ট স্টোরিগুলি ঘ...      2

In [10]:
NAG = data_test[(data_test['Sub-task A']=='NAG')]
OAG = data_test[(data_test['Sub-task A']=='CAG')]
CAG = data_test[(data_test['Sub-task A']=='OAG')]

In [11]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG.head()

Text  Sub-task A
0   Ek dom sothik kotha bolecho jhekane theka uthe...           0
2                                    DARUN AKDOM THIK           0
5                                     Kadam thik dada           0
12                               Supar bolla cho dada           0
15                                              Pagle           0

In [12]:
data_test = pd.concat([NAG,
                  OAG,
                  CAG
                 ],axis=0)

In [14]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for explainer
#from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
#import transformers

In [15]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=53854cfd636d29bcb1c88a6f29ac6feff61ccd98617b9be798c1c1c288cc7cd5
  Stored in directory: /tmp/pip-ephem-wheel-cache-kpqurno9/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [16]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [17]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

In [18]:
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

In [21]:
data['vec'] = data['Text'].apply(lambda x: get_vec(x))

In [22]:
data_test['vec'] = data_test['Text'].apply(lambda x: get_vec(x))

In [23]:
X = data['vec'].to_numpy()
X = X.reshape(-1, 1)

In [24]:
P = data_test['vec'].to_numpy()
P = P.reshape(-1, 1)

In [25]:
X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)

In [26]:
P = np.concatenate(np.concatenate(P, axis = 0), axis = 0).reshape(-1, 300)

In [29]:
y = data['label']

In [30]:
Q = data_test['Sub-task A']

In [31]:
X_train = X
X_test = P
y_train = y 
y_test = Q



In [32]:
clf = LogisticRegression(solver='liblinear')

In [33]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
y_pred = clf.predict(X_test)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.92      0.73       522
           1       0.39      0.10      0.16       218
           2       0.65      0.30      0.41       217

    accuracy                           0.59       957
   macro avg       0.55      0.44      0.43       957
weighted avg       0.56      0.59      0.53       957

